In [114]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

import pandas as pd
import numpy as np

In [115]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv('gender_submission.csv')

In [116]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 檢查資料缺失值

In [117]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [118]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [119]:
train.describe()

C:\Users\kevin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [120]:
test.describe()

C:\Users\kevin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,NaN,0.000000,0.000000,NaN
50%,1100.500000,3.000000,NaN,0.000000,0.000000,NaN
75%,1204.750000,3.000000,NaN,1.000000,0.000000,NaN
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## 選擇想要分析的特徵

In [121]:
selected_features = ['Pclass', 'Age', 'Fare', 'Sex',"SibSp"]

In [122]:
X_train = train[selected_features]

In [123]:
y_train = train['Survived']

In [124]:
X_test = test[selected_features]

## 處理缺失值

In [125]:
X_train['Age'].fillna(X_train['Age'].mean()+5, inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)

C:\Users\kevin\Anaconda3\lib\site-packages\pandas\core\generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


## 把類別變數轉成Dummy

In [126]:
X_train = pd.get_dummies(X_train)

In [127]:
X_test_Sex = pd.get_dummies(X_test["Sex"])

In [128]:
X_test = X_test.join(X_test_Sex)

In [129]:
X_test = X_test.drop("Sex", axis=1)

## 建立Keras Sequential模型

In [130]:
model = Sequential()

## Add Hidden Layer -1

In [131]:
model.add(Dense(units = 50, input_dim = 6, 
                kernel_initializer = 'uniform', 
                bias_initializer='zeros', 
                activation = 'relu'))

## Add Hidden Layer -2

In [132]:
model.add(Dense(units = 20, 
                kernel_initializer = 'uniform',
                bias_initializer='zeros', 
                activation = 'relu'))

## Add Output Layer

In [133]:
model.add(Dense(units = 1, 
                kernel_initializer = 'uniform',
                activation = 'sigmoid'))

## 訓練模型

In [137]:
model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam', 
              metrics=['accuracy'])

In [138]:
model.fit(X_train, y_train, validation_split=0.5, epochs=50)

Train on 445 samples, validate on 446 samples
Epoch 1/50
445/445 [==============================] - 1s 2ms/step - loss: 0.6913 - acc: 0.5888 - val_loss: 0.6854 - val_acc: 0.6682
Epoch 2/50
445/445 [==============================] - 0s 173us/step - loss: 0.6828 - acc: 0.6247 - val_loss: 0.6713 - val_acc: 0.6659
Epoch 3/50
445/445 [==============================] - 0s 146us/step - loss: 0.6671 - acc: 0.6472 - val_loss: 0.6445 - val_acc: 0.6906
Epoch 4/50
445/445 [==============================] - 0s 120us/step - loss: 0.6472 - acc: 0.6629 - val_loss: 0.6120 - val_acc: 0.6928
Epoch 5/50
445/445 [==============================] - 0s 140us/step - loss: 0.6297 - acc: 0.6674 - val_loss: 0.5947 - val_acc: 0.7040
Epoch 6/50
445/445 [==============================] - 0s 117us/step - loss: 0.6229 - acc: 0.6697 - val_loss: 0.5798 - val_acc: 0.7040
Epoch 7/50
445/445 [==============================] - 0s 149us/step - loss: 0.6177 - acc: 0.6697 - val_loss: 0.5763 - val_acc: 0.7085
Epoch 8/50
445/445

In [139]:
scores = model.evaluate(X_train, y_train)
scores[1]

891/891 [==============================] - 0s 41us/step


0.7946127943452093

## 利用建構模型進行預測

In [140]:
survived_predict = model.predict(X_test)
survived_predict = np.array(survived_predict)
survived_predict = np.round(survived_predict)
survived_predict

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],

## 輸出結果

In [141]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict[0:418,0]})

In [142]:
submission.to_csv('submissionbykeras.csv', index=False)